<a href="https://colab.research.google.com/github/ThomasLincoln/Faculdade/blob/main/Introducao_GPU_secao3Parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/lesc-ufv/wscad2020/master/figures/logo-wscad.png" alt="wscad2020" width="90.5"/>
<img src="https://raw.githubusercontent.com/lesc-ufv/wscad2020/master/figures/logo-dpi.jpg" alt="dpi ufv" width="64"/>
<img src="https://raw.githubusercontent.com/lesc-ufv/wscad2020/master/figures/logo-lesc.jpg" alt="lesc ufv" width="41.5"/>
<img src="https://raw.githubusercontent.com/lesc-ufv/wscad2020/master/figures/logo-fapemig.jpg" alt="fapemig" width="73"/>
<img src="https://raw.githubusercontent.com/lesc-ufv/wscad2020/master/figures/logo-cnpq.jpg" alt="cnpq" width="133"/>
<img src="https://raw.githubusercontent.com/lesc-ufv/wscad2020/master/figures/logo-capes.jpg" alt="capes" width="59"/>
<img src="https://raw.githubusercontent.com/lesc-ufv/wscad2020/master/figures/logo-ufv.png" alt="ufv" width="79"/>
<img src="https://raw.githubusercontent.com/lesc-ufv/wscad2020/master/figures/logo-google.jpg" alt="google" width="105"/>
<img src="https://raw.githubusercontent.com/lesc-ufv/wscad2020/master/figures/logo-nvidia.png" alt="Nvidia" width="59.3"/>

< [Seção 2 - Introdução ao Colab](https://colab.research.google.com/drive/1YgB_zCS7VY9hF8Zs1Du5YUyF5z7sEX0w?usp=sharing) | [Índice Geral](https://colab.research.google.com/drive/1eYk9GFgFsLnfxQVMHphMyztFc0GIuZYS#scrollTo=vQ_qJrHy7Omj) | [Seção 3 - Introdução a GPU Parte II](https://colab.research.google.com/drive/1Mh8ByAyh0iOBtZwcyzCq2QBW1teVUJ9s?usp=sharing) >

#Seção 3: Introdução a GPU - Parte I

Primeiro exemplo de como executar um código GPU no Colab.

## **Programação em GPU no Ambiente Google Colaboratory**

Ricardo Ferreira, Michael Canesche, Westerley Carvalho. Universidade Federal de Viçosa - ricardo@ufv.br

Minicurso do [Simpósio de Sistemas Computacionais de Alto Desempenho](http://wscad.sbc.org.br/2020/chamada-minicursos.html) (WSCAD 2020). [**Links para video, texto completo e slides**](https://github.com/lesc-ufv/wscad2020/blob/master/links/readme.md)

## **Configurando o Google Lab (colab)**
Executar o comando abaixo para permitir a execução dos códigos.

In [ ]:
!pip install git+https://github.com/lesc-ufv/cad4u.git &> /dev/null
!git clone https://github.com/lesc-ufv/cad4u &> /dev/null
%load_ext plugin

# Introdução à GPU

O comando **!nvidia-smi** nos mostra as informações sobre a GPU disponibilizada pelo Google Colaboratory no sua sessão do Colab. Em geral, as placas que são oferecidas são uma K80 ou uma T4.




In [ ]:
!nvidia-smi

Tue Sep  6 11:25:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Kernels

A grande vantagem de se utilizar uma GPU é a possibilidade de se realizar várias computações simultaneamente, enquanto em uma CPU essas mesmas computações seriam realizadas de forma sequencial. O trecho de código a ser executado em paralelo é especificado utilizando funções, normalmente  referenciadas pelo termo ***kernel***.

O exemplo, a seguir, apresenta um kernel simples e uma função normal da linguagem C. Note que ambos realizam a mesma tarefa, imprimir "Hello World" na tela, mas uma é executada na GPU e a outra na CPU. O que diferencia a função da GPU é a palavra chave **\_\_global\_\_** antes da declaração da função. Outro aspecto é a chamada da função que especifica a quantidade de cópias (ou threads) que serão executadas. O exemplo executa apenas 1 thread.

In [ ]:
%%gpu
#include <stdio.h>

//#A Keyword __global__ indica que esta função é um Kernel

__global__ void helloFromGPU(){
    printf("Hello World from GPU!!!\n");
    return;
}

void helloFromCPU(){
    printf("Hello World from CPU!\n");
    return;
}

int main(){
    //Chamada do Kernel com apenas 1 thread em 1 bloco.
    helloFromGPU<<<1,1>>>();

    //Chamada da função CPU
    helloFromCPU();


    cudaDeviceReset();
   cudaDeviceSynchronize();
    return 0;
}

Hello World from CPU!
Hello World from GPU!!!



# Blocos e Threads

Note que ao chamar o kernel, foram passados os valores <<<1,1>>> que representam respectivamente o número de threads e o número de blocos a serem usados pela GPU.
>                    helloFromGPU<<<#blocos, #threads>>>

*   As **threads** representam cada instância de execução paralela do kernel e são agrupadas em **blocos**. O número de threads é do tipo *dim3* que possibilita a identificação das threads em arranjos 1D, 2D e 3D. O número máximo de threads por bloco é limitado e pode ser 512, 1024 ou 2048 dependendo da placa.
*   Os **blocos** são organizados em um **grid** de 1 ou 2 dimensões. Todos os blocos de um grid terão o mesmo tamanho.
Um ou mais blocos serão alocados para executar em multiprocessadores (**Stream Multiprocessor ou SM**). Quando um bloco é escalonado em um SM irá permanecer no SM até terminar sua execução.  **Os blocos e seus threads são executados em ordem aleatória**. A quantidade de blocos que executa simultaneamente em um SM depende da quantidade de recursos do SM e da quantidade de recursos requisitados pelo bloco (número de registradores e quantidade de memória compartilhada), além é claro do limite máximo definido para cada arquitetura de GPU.

No trecho de código a seguir, vamos novamente usar o kernel helloFromGPU() porém agora vamos lançar mais threads. Para que possamos identificar dentro do kernel qual thread e qual bloco estão executando, vamos usar as variáveis built-in **threadIdx.x** e **blockIdx.x**. Para **N** blocos de **M** threads, os blocos são rotulados de 0 a N-1 e os threads dentro de cada bloco de 0 a M-1.


In [ ]:
%%gpu
#include <stdio.h>

//Kernel (indicado pela keyword __global__)

void helloFromCPU(){
    printf("Hello World from CPU!\n");
    return;
}

__global__ void helloFromGPU(){
    printf("Hello World from GPU! Block number: %d Thread number: %d\n", blockIdx.x ,threadIdx.x);
    return;
}

int main(){
    printf("Hello World from CPU!\n");

    //Chamada do Kernel
    helloFromGPU<<<2,64>>>();

    cudaDeviceReset();
    cudaDeviceSynchronize();
    return 0;
}

## Referência Complementar

Recomendamos o trecho do livro [Learn Cuda Programming](https://www.packtpub.com/product/learn-cuda-programming/9781788996242).

# Considerações Finais

Este laboratório teve o objetivo de executar uma função na GPU e introduzir a organização dos threads e blocos. As próximas sessões irão aprofundar nestes tópicos e no modelo de execução da GPU. Se você tem alguma sugestão, enviar uma mensagem para o email ricardo@ufv.br

# Nvidia T4

[Fonte Wiki](https://en.wikipedia.org/wiki/CUDA):

![](https://tvm.d2l.ai/_images/gpu_sm.svg)
- capability 7.5
- Máximo Blocos dim x = $2^{31}-1=2Giga= 2$ bilhões
- Máximo Blocos dim y e z = 65535
- Máximo número de Threads por bloco = 1024
- Máximo número de Blocos residentes em um multiprocessador = 16
- Máximo de registradores por bloco 64k
- Máximo de registradores por multiprocessador 64k
- Máximo de registradores por threas 255
- Multiprocessadores com 64 cores, 40 multiprocessadores = 40 SM com 64 cores = 2560 cores


![](https://tvm.d2l.ai/_images/gpu_t4.svg)

# Exercícios

In [ ]:
#@title 1. Experimente variar a quantidade de blocos e a quantidade de threads lançadas pelo kernel. O que você pode dizer sobre a ordem de execução dos blocos e das threads?

resposta = 'A ordem das threads e blocos e aleatoria' #@param ["A ordem das threads e sempre crescente e a de blocos tambem","A ordem das threads e blocos e aleatoria","A ordem das threads e aleatoria e a de blocos e sempre crescente","A ordem das threads e aleatória e a de blocos e sempre crescente"]

if (resposta == 'A ordem das threads e blocos e aleatoria' ):
    print("Correto !")
else:
    print("Tente novamente.")

Correto !


In [ ]:
#@title 2. Dentro do kernel, inclua uma chamada para a função helloFromCPU(). O que acontece quando você executa o programa? Porque isso acontece?

resposta = 'O programa nao funciona porque nao se pode chamar uma funcao dentro de um kernel' #@param ["O programa nao funciona porque nao se pode chamar uma funcao dentro de um kernel","O programa funciona normalmente pois a funcão helloFromCPU() pode ser acessada pelo device","O kernel nao consegue executar helloFromCPU() pois ela esta fora do seu escopo"]

if (resposta == 'O kernel nao consegue executar helloFromCPU() pois ela esta fora do seu escopo' ):
    print("Correto !")
else:
    print("Tente novamente.")

3. O código abaixo [02_First_Kernel](https://github.com/eegkno/CUDA_by_practice/blob/master/02_First_kernel/Source_code/02simple_kernel2/02simple_kernel2.cu) do repositório [CUDA by practice](https://github.com/eegkno/CUDA_by_practice) de Edgar Garcia Cano apresenta um exemplo de uma função que realiza uma soma na GPU e retorna o valor para CPU. Execute este exemplo e observe que outras funções foram incluídas. Este tema será tratado no próximo laboratório.

In [ ]:
%%gpu
#include <assert.h>
#include <stdio.h>

__global__ void addKernel(int a, int b, int* c) {
    *c = a + b;
}

int main(void) {
    int h_c;
    int* d_c;
    const int C_BYTES = 1 * sizeof(int);

    // Save memory
    cudaMalloc((void**)&d_c, C_BYTES);

    // Call the kernel
    addKernel<<<1, 1>>>(2, 7, d_c);

    // Copy memory from Device to Host
    cudaMemcpy(&h_c, d_c, sizeof(int), cudaMemcpyDeviceToHost);

    assert(2 + 7 == h_c);
    printf("-: successful execution :- result = %d \n",h_c);

    // Free device memory
    cudaFree(d_c);

    return 0;
}

-: successful execution :- result = 9 



## Exercícios suplementares I

A seguir alguns exemplos do capítulo adaptados do Livro Cuda C Programming, [disponíveis aqui](https://media.wiley.com/product_ancillary/29/11187393/DOWNLOAD/CodeSamples.zip)

In [ ]:
%%gpu
// O código abaixo mostra como imprimir detalhadamente as características da GPU

#include <cuda_runtime.h>
#include <stdio.h>

/*
 * Display a variety of information on the first CUDA device in this system,
 * including driver version, runtime version, compute capability, bytes of
 * global memory, etc.
 */

int main(int argc, char **argv)
{
    printf("%s Starting...\n", argv[0]);

    int deviceCount = 0;
    cudaGetDeviceCount(&deviceCount);

    if (deviceCount == 0)
    {
        printf("There are no available device(s) that support CUDA\n");
    }
    else
    {
        printf("Detected %d CUDA Capable device(s)\n", deviceCount);
    }

    int dev = 0, driverVersion = 0, runtimeVersion = 0;
    cudaSetDevice(dev);
    cudaDeviceProp deviceProp;
   cudaGetDeviceProperties(&deviceProp, dev);
    printf("Device %d: \"%s\"\n", dev, deviceProp.name);

    cudaDriverGetVersion(&driverVersion);
    cudaRuntimeGetVersion(&runtimeVersion);
    printf("  CUDA Driver Version / Runtime Version          %d.%d / %d.%d\n",
           driverVersion / 1000, (driverVersion % 100) / 10,
           runtimeVersion / 1000, (runtimeVersion % 100) / 10);
    printf("  CUDA Capability Major/Minor version number:    %d.%d\n",
           deviceProp.major, deviceProp.minor);
    printf("  Total amount of global memory:                 %.2f GBytes (%llu "
           "bytes)\n", (float)deviceProp.totalGlobalMem / pow(1024.0, 3),
           (unsigned long long)deviceProp.totalGlobalMem);
    printf("  GPU Clock rate:                                %.0f MHz (%0.2f "
           "GHz)\n", deviceProp.clockRate * 1e-3f,
           deviceProp.clockRate * 1e-6f);
    printf("  Memory Clock rate:                             %.0f Mhz\n",
           deviceProp.memoryClockRate * 1e-3f);
    printf("  Memory Bus Width:                              %d-bit\n",
           deviceProp.memoryBusWidth);

    if (deviceProp.l2CacheSize)
    {
        printf("  L2 Cache Size:                                 %d bytes\n",
               deviceProp.l2CacheSize);
    }

    printf("  Max Texture Dimension Size (x,y,z)             1D=(%d), "
           "2D=(%d,%d), 3D=(%d,%d,%d)\n", deviceProp.maxTexture1D,
           deviceProp.maxTexture2D[0], deviceProp.maxTexture2D[1],
           deviceProp.maxTexture3D[0], deviceProp.maxTexture3D[1],
           deviceProp.maxTexture3D[2]);
    printf("  Max Layered Texture Size (dim) x layers        1D=(%d) x %d, "
           "2D=(%d,%d) x %d\n", deviceProp.maxTexture1DLayered[0],
           deviceProp.maxTexture1DLayered[1], deviceProp.maxTexture2DLayered[0],
           deviceProp.maxTexture2DLayered[1],
           deviceProp.maxTexture2DLayered[2]);
    printf("  Total amount of constant memory:               %lu bytes\n",
           deviceProp.totalConstMem);
    printf("  Total amount of shared memory per block:       %lu bytes\n",
           deviceProp.sharedMemPerBlock);
    printf("  Total number of registers available per block: %d\n",
           deviceProp.regsPerBlock);
    printf("  Warp size:                                     %d\n",
           deviceProp.warpSize);
    printf("  Maximum number of threads per multiprocessor:  %d\n",
           deviceProp.maxThreadsPerMultiProcessor);
    printf("  Maximum number of threads per block:           %d\n",
           deviceProp.maxThreadsPerBlock);
    printf("  Maximum sizes of each dimension of a block:    %d x %d x %d\n",
           deviceProp.maxThreadsDim[0],
           deviceProp.maxThreadsDim[1],
           deviceProp.maxThreadsDim[2]);
    printf("  Maximum sizes of each dimension of a grid:     %d x %d x %d\n",
           deviceProp.maxGridSize[0],
           deviceProp.maxGridSize[1],
           deviceProp.maxGridSize[2]);
    printf("  Maximum memory pitch:                          %lu bytes\n",
           deviceProp.memPitch);

    exit(0);
}

/content/code.out Starting...
Detected 1 CUDA Capable device(s)
Device 0: "Tesla K80"
  CUDA Driver Version / Runtime Version          11.2 / 11.0
  CUDA Capability Major/Minor version number:    3.7
  Total amount of global memory:                 11.17 GBytes (11996954624 bytes)
  GPU Clock rate:                                824 MHz (0.82 GHz)
  Memory Clock rate:                             2505 Mhz
  Memory Bus Width:                              384-bit
  L2 Cache Size:                                 1572864 bytes
  Max Texture Dimension Size (x,y,z)             1D=(65536), 2D=(65536,65536), 3D=(4096,4096,4096)
  Max Layered Texture Size (dim) x layers        1D=(16384) x 2048, 2D=(16384,16384) x 2048
  Total amount of constant memory:               65536 bytes
  Total amount of shared memory per block:       49152 bytes
  Total number of registers available per block: 65536
  Warp size:                                     32
  Maximum number of threads per multiprocessor:  204

In [ ]:
%%gpu
/* Dividir 1024 elementos em um ou mais blocos.
Lembrando que todos os blocos de um determinado grid terão o mesmo tamanho.
*/
#include <cuda_runtime.h>
#include <stdio.h>

/*
 * Demonstrate defining the dimensions of a block of threads and a grid of
 * blocks from the host.
 */

int main(int argc, char **argv)
{
    // define total data element
    int nElem = 1024;

    // define grid and block structure
    dim3 block (1024);
    dim3 grid  ((nElem + block.x - 1) / block.x);
    printf("grid.x %d block.x %d \n", grid.x, block.x);

    // reset block
    block.x = 512;
    grid.x  = (nElem + block.x - 1) / block.x;
    printf("grid.x %d block.x %d \n", grid.x, block.x);

    // reset block
    block.x = 256;
    grid.x  = (nElem + block.x - 1) / block.x;
    printf("grid.x %d block.x %d \n", grid.x, block.x);

    // reset block
    block.x = 128;
    grid.x  = (nElem + block.x - 1) / block.x;
    printf("grid.x %d block.x %d \n", grid.x, block.x);

    // reset device before you leave
   cudaDeviceReset();

    return(0);
}


grid.x 1 block.x 1024 
grid.x 2 block.x 512 
grid.x 4 block.x 256 
grid.x 8 block.x 128 



## Material Suplementar

* Recomendamos o mini-tutorial [do Jonathan Hui](https://jhui.github.io/2017/03/06/CUDA/)

< [Seção 2 - Introdução ao Colab](https://colab.research.google.com/drive/1YgB_zCS7VY9hF8Zs1Du5YUyF5z7sEX0w?usp=sharing) | [Índice Geral](https://colab.research.google.com/drive/1eYk9GFgFsLnfxQVMHphMyztFc0GIuZYS#scrollTo=vQ_qJrHy7Omj) | [Seção 3 - Introdução a GPU Parte II](https://colab.research.google.com/drive/1Mh8ByAyh0iOBtZwcyzCq2QBW1teVUJ9s?usp=sharing) >